In [1]:
from sentence_transformers import SentenceTransformer, util
import os
# from utils import freq
import re
import pandas as pd
import numpy as np

d:\Program\Anaconda\envs\CP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [123]:
df_posts = pd.read_csv('all_posts.csv',  encoding='utf-8', dtype={'user_id':str})
# 把nan变为None
# df_posts = df_posts.where(pd.notnull(df_posts), None)

df_posts.shape

(15664, 19)

In [124]:
# 按照日期限制范围
def get_post_date_range(df,start,end,cluster = None):
    if cluster:
        df_cluster = df[df['cluster'] == cluster]
    else:
        df_cluster = df
    df_date = df_cluster[(df_cluster['publish_time'] >= start) & (df_cluster['publish_time'] <= end)]
    return df_date
# 按照cluster统计每组的数量
def get_post_gorup_count(df,group = 'cluster'):
    df_cluster_count = df.groupby(group).size().reset_index(name='count')
    return df_cluster_count
get_post_gorup_count(df_posts)
# 根据文本列表查找
def get_post_by_text_list(df,text_list):
    df_text = df[df['text'].str.contains('|'.join(text_list))]
    return df_text
# 把df_snccll的时间随机到'2021-04-20','2021-04-29'，注意要带上时分秒
def random_date(df,start,end):
    df['publish_time'] = pd.to_datetime(df['publish_time'], format='%Y-%m-%d %H:%M:%S')
    df['publish_time'] = df['publish_time'].apply(lambda x: x.replace(year=2021, month=4, day=np.random.randint(20, 30)))
    df['publish_time'] = df['publish_time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    return df
# 按照日期限制范围
def get_post_date_out_range(df,start,end,cluster = None):
    if cluster:
        df_cluster = df[df['cluster'] == cluster]
    else:
        df_cluster = df
    df_date = df_cluster[(df_cluster['publish_time'] < start) | (df_cluster['publish_time'] > end)]
    return df_date

# japan_dead_fish

In [125]:
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(1, 19)

In [126]:
# 查找 2023.12.1 -> 2023.12.10之间，twitter上query为japan dead fish的
df_data_twitter = df_posts[df_posts['from'] == 'twitter']
df_data_twitter = get_post_date_range(df_data_twitter,'2023-12-01','2023-12-10')
df_data_twitter = df_data_twitter[df_data_twitter['query']== "Japan dead fish"]
df_data_twitter.shape
# 把这些data的cluster设置为japan_dead_fish
df_data_twitter['cluster'] = 'japan_dead_fish'
# 更新这些数据在df_posts
df_posts.update(df_data_twitter)
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(12, 19)

In [127]:
df_data_weibo = df_posts[df_posts['from'] == 'weibo']
df_data_weibo = get_post_date_range(df_data_weibo,'2023-12-01','2023-12-10')
df_data_weibo = df_data_weibo[df_data_weibo['query'] == "w9"]
df_data_weibo.shape
# 把这些data的cluster设置为japan_dead_fish
df_data_weibo['cluster'] = 'japan_dead_fish'
# 更新这些数据在df_posts
df_posts.update(df_data_weibo)
df_posts[df_posts['cluster'] =='japan_dead_fish'].shape

(13, 19)

In [128]:
# 保存
df_posts.to_csv('all_posts.csv',index=False, encoding='utf-8')

In [131]:
df_accounts = pd.read_csv('all_accounts.csv',  encoding='utf-8', dtype={'user_id':str})
# 去除nan为None
df_accounts = df_accounts.where(pd.notnull(df_accounts), None)


In [132]:
# 查看'validation'的类型都有什么
df_accounts['validation'].unique()


array(['True', 'False', 'validation', None], dtype=object)

In [133]:
df_accounts[df_accounts['validation'] == 'validation']

,user_id,avatar,screen_name,name,description,fan,validation,type,verified_reason,screen_name_trans,description_trans,verified_reason_trans
6221,user_id,avatar,screen_name,name,description,fan,validation,type,None,screen_name,description,None


In [134]:
df_accounts['verified_reason_trans'].unique()

array(['Internet technology blogger headline author',
       '2023 Weibo Annual New Knowledge Blogger by Resident Journalist in the United States',
       None, ...,
       'Public Security Bureau of Xuancheng City, Anhui Province',
       "Official Weibo account of Lianyungang Municipal People's Government",
       'Fujian Provincial Committee'], dtype=object)